In [1]:
import tensorflow as tf
import numpy as np
import sys
sys.path.append("..") # Adds higher directory to python modules path.
import branchingdnn as branching
from branchingdnn.utils import * 
from branchingdnn.profiler import model_profiler as profiler
import h5py
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, Concatenate
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D

Using TensorFlow backend.


In [6]:
x = tf.keras.models.load_model('../models/alexNetv6.hdf5', custom_objects={'confidenceScore': confidenceScore,
                                                                         'unconfidence': unconfidence,
                                                                         'confidenceDifference': confidenceDifference,
                                                                         'BranchEndpoint': branching.branches.branch.BranchEndpoint,
                                                                         'FeatureDistillation': branching.branches.branch.FeatureDistillation})
x.summary()

Model: "alexnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0   

In [5]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model
x.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001,momentum=0.9), metrics=['accuracy'])
# q_aware stands for for quantization aware.
q_aware_model = quantize_model(x)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

q_aware_model.summary()

# model = keras.Model(inputs=inputs, outputs=[x,branchLayer,branchLayer2], name="alexnet")
# model = keras.Model(inputs=inputs, outputs=[x], name="alexnet")
# # model.save("models/alexNetv6.hdf5")


# # checkpoint = keras.callbacks.ModelCheckpoint("models/alexNetv6.hdf5", monitor='val_loss',verbose=1,save_best_only=True, mode='auto',period=1)

# model.summary()

RuntimeError: Layer batch_normalization:<class 'tensorflow.python.keras.layers.normalization_v2.BatchNormalization'> is not supported. You can quantize this layer by passing a `tfmot.quantization.keras.QuantizeConfig` instance to the `quantize_annotate_layer` API.